In [ ]:
#| default_exp utils

# Utils

> App setup and utility functions

In [ ]:
#| export
from fastcore.utils import *
from fasthtml.common import *
from fasthtml.common import fast_app as ori_fast_app, FastHTML as ori_FastHTML
from fasthtml.jupyter import *
from fastlucide import *
from fastcore.meta import delegates

### Basecoat headers

In [ ]:
#| export
# Additional script needed for sliders
slider_script = Script("""
const updateSlider = (el) => {
    const min = parseFloat(el.min || 0);
    const max = parseFloat(el.max || 100);
    const value = parseFloat(el.value);
    const percent = (max === min) ? 0 : ((value - min) / (max - min)) * 100;
    el.style.setProperty('--slider-value', `${percent}%`);
};
""")

In [ ]:
#| export
# For some reason the text tailwind classes are not being properly generated
text_css = Style("""
.text-muted-foreground { color: var(--muted-foreground); }
.text-foreground { color: var(--foreground); }
.hover\\:text-foreground:hover { color: var(--foreground); }
.bg-accent { background-color: var(--accent); }
""")
theme_script = Script("""
(() => {
  const stored = localStorage.getItem('themeMode');
  const dark = stored ? stored === 'dark' : matchMedia('(prefers-color-scheme: dark)').matches;
  if (dark) document.documentElement.classList.add('dark');
  
  document.addEventListener('basecoat:theme', (e) => {
    const mode = e.detail?.mode || (document.documentElement.classList.contains('dark') ? 'light' : 'dark');
    const isDark = mode === 'dark';
    document.documentElement.classList.toggle('dark', isDark);
    localStorage.setItem('themeMode', isDark ? 'dark' : 'light');
  });
})();
""")

ss = SvgSprites('l-')
Icon = ss # Renaming for more clarity when it is used

deps = {
    'scripts': [
        'https://cdn.jsdelivr.net/npm/@tailwindcss/browser@4',
        'https://cdn.jsdelivr.net/npm/lit@3/dist/index.js',
        'https://cdn.jsdelivr.net/npm/basecoat-css@0.3.6/dist/js/all.min.js'
    ],
    'links': [
        'https://cdn.jsdelivr.net/npm/basecoat-css@0.3.6/dist/basecoat.cdn.min.css'
    ]
}

def make_hdrs(deps):
    scripts = tuple(Script(src=url) for url in deps['scripts'])
    links = tuple(Link(rel='stylesheet', href=url) for url in deps['links'])
    return (theme_script, ss) + scripts + links + (text_css, slider_script)

basecoat_hdrs = make_hdrs(deps)

### Fasthtml

In [ ]:
#| export
@delegates(ori_FastHTML, keep=True, but=["pico"])
def FastHTML(hdrs=None, pico=False, **kwargs):
    hdrs = basecoat_hdrs + (hdrs or ())
    return ori_FastHTML(hdrs=hdrs, pico=pico, **kwargs)

In [ ]:
FastHTML

<function __main__.FastHTML(hdrs=None, pico=False, *, debug=False, routes=None, middleware=None, title: str = 'FastHTML page', exception_handlers=None, on_startup=None, on_shutdown=None, lifespan=None, ftrs=None, exts=None, before=None, after=None, surreal=True, htmx=True, default_hdrs=True, sess_cls=<class 'starlette.middleware.sessions.SessionMiddleware'>, secret_key=None, session_cookie='session_', max_age=31536000, sess_path='/', same_site='lax', sess_https_only=False, sess_domain=None, key_fname='.sesskey', body_wrap=<function noop_body>, htmlkw=None, nb_hdrs=False, canonical=True, **kwargs)>

In [ ]:
app = FastHTML(session_cookie="mysession")
rt = app.route

In [ ]:
#| export
def get_preview(app=None): 
    if not app: app = FastHTML(session_cookie="mysession")
    return partial(HTMX, app=app, host=None, port=None)
p = get_preview()

In [ ]:
c = Div(
    Button('Hey there', cls='btn-outline'),
)
p(c)

In [ ]:
#| export
def slugify(s):
    return s.lower().replace(" ", "-")

In [ ]:
slugify("This content")

'this-content'

In [ ]:
#| export
# To easily preview items in a larger container
def Window(*args, cls="h-96"):
    return Div(*args, cls="w-full flex flex-col items-center justify-center {cls}")

In [ ]:
#| export
def pw(*args, **kwargs):
    return p(Div(Window(*args, **kwargs), cls="h-100 w-full flex flex-col justify-center items-center"))

In [ ]:
pw(c)

In [ ]:
#| export
class VEnum(Enum):
    def __str__(self): return self.value
    def __add__(self, b): return f"{self.value} {b}"
    def __radd__(self, a): return f"{a} {self.value}"

In [ ]:
class TestEnum(VEnum):
    test1 = "testing-one"
    test2 = "testing-two"

In [ ]:
TestEnum.test1 + TestEnum.test2

'testing-one testing-two'

In [ ]:
"h2" + TestEnum.test1

'h2 testing-one'